<a href="https://colab.research.google.com/github/mohamedalaaaz/EmailModule/blob/main/space22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
// solar_mc.cpp
// C++17 standalone Monte Carlo solar-wind -> spacecraft Delta-V pipeline
// Outputs CSVs: p_dyn_percentiles.csv, e_rec_percentiles.csv, delta_v_hist.csv, delta_v_time_percentiles.csv

#include <bits/stdc++.h>
using namespace std;

// ---------- Utility functions ----------
double rand_normal(std::mt19937 &rng, double mu, double sigma) {
    std::normal_distribution<double> dist(mu, sigma);
    return dist(rng);
}

vector<double> percentile_vector(const vector<vector<double>>& data, double p) {
    // data: [N_samples][T] ; returns vector length T of p-th percentile
    int N = data.size();
    if (N == 0) return {};
    int T = data[0].size();
    vector<double> out(T);
    vector<double> tmp(N);
    for (int t = 0; t < T; ++t) {
        for (int i = 0; i < N; ++i) tmp[i] = data[i][t];
        int k = max(0, min(N - 1, (int)floor((p/100.0) * (N - 1) + 0.5)));
        nth_element(tmp.begin(), tmp.begin() + k, tmp.end());
        out[t] = tmp[k];
    }
    return out;
}

vector<double> median_vector(const vector<vector<double>>& data) { return percentile_vector(data, 50.0); }

void write_time_csv(const string &fname, const vector<double>& t, const vector<double>& med,
                    const vector<double>& p10, const vector<double>& p90, const string &ylabel) {
    ofstream f(fname);
    f << "time," << "median," << "p10," << "p90\n";
    int T = t.size();
    for (int i = 0; i < T; ++i) {
        f << t[i] << "," << med[i] << "," << p10[i] << "," << p90[i] << "\n";
    }
    f.close();
}

void write_delta_v_hist_csv(const string &fname, const vector<double>& delta_v_ms) {
    ofstream f(fname);
    f << "delta_v_m_s\n";
    for (double v : delta_v_ms) f << v << "\n";
    f.close();
}

void write_delta_v_time_percentiles(const string &fname, const vector<double>& t,
                                     const vector<double>& med, const vector<double>& p10, const vector<double>& p90) {
    write_time_csv(fname, t, med, p10, p90, "delta_v");
}

// ---------- Main simulation ----------
int main() {
    // Constants
    const double MU0 = 4.0 * M_PI * 1e-7;
    const double RE = 6.371e6;
    const double m_p = 1.6726e-27;

    // Simulation grid
    const double hours = 72.0;
    const double dt_hr = 1.0;
    const double dt = dt_hr * 3600.0;
    vector<double> t;
    for (double tt = 0.0; tt <= hours + 1e-9; tt += dt_hr) t.push_back(tt);
    const int T = (int)t.size();

    // Background parameters
    const double v_bg = 450e3;
    const double n_bg = 5.0; // particles/cm^3
    const double Bz_bg = -2e-9;
    const double By_bg = 1e-9;

    // Nominal pulse params
    const double pulse_center = 24.0;
    const double pulse_width = 6.0;
    const double pulse_amp_v_nom = 800e3;
    const double pulse_amp_n_nom = 50.0;
    const double pulse_amp_Bz_nom = -15e-9;

    // Monte Carlo
    const int N_mc = 300;
    std::mt19937 rng(42);
    const double v_sigma = 0.2 * pulse_amp_v_nom;
    const double n_sigma = 0.3 * pulse_amp_n_nom;
    const double Bz_sigma = 0.3 * fabs(pulse_amp_Bz_nom);

    const double A_over_m_sc = 0.02;
    const double C_d = 2.0;

    auto gauss_profile = [&](double tt)->double {
        double x = (tt - pulse_center) / pulse_width;
        return exp(-0.5 * x * x);
    };

    // Reserve containers
    vector<vector<double>> Pdyn_all; Pdyn_all.reserve(N_mc);
    vector<vector<double>> Erec_all; Erec_all.reserve(N_mc);
    vector<vector<double>> delta_v_time_all; delta_v_time_all.reserve(N_mc);
    vector<double> delta_v_final; delta_v_final.reserve(N_mc);

    // Run MC
    for (int i = 0; i < N_mc; ++i) {
        double amp_v = rand_normal(rng, pulse_amp_v_nom, v_sigma);
        double amp_n = rand_normal(rng, pulse_amp_n_nom, n_sigma);
        double amp_Bz = rand_normal(rng, pulse_amp_Bz_nom, Bz_sigma);

        amp_v = max(0.0, amp_v);
        amp_n = max(0.0, amp_n);

        vector<double> Pdyn_t(T);
        vector<double> Erec_t(T);
        vector<double> delta_v_time(T);

        double delta_v_accum = 0.0;
        for (int ti = 0; ti < T; ++ti) {
            double tt = t[ti];
            double v_t = v_bg + amp_v * gauss_profile(tt);
            double n_t = n_bg + amp_n * gauss_profile(tt);
            double Bz_t = Bz_bg + amp_Bz * gauss_profile(tt);
            double By_t = By_bg;

            double n_si = n_t * 1e6 * m_p;
            double Pdyn = n_si * v_t * v_t; // Pa
            double Erec = v_t * sqrt(By_t*By_t + Bz_t*Bz_t);

            Pdyn_t[ti] = Pdyn;
            Erec_t[ti] = Erec;

            double a_drag = C_d * A_over_m_sc * Pdyn; // m/s^2
            delta_v_accum += a_drag * dt;
            delta_v_time[ti] = delta_v_accum;
        }

        Pdyn_all.push_back(move(Pdyn_t));
        Erec_all.push_back(move(Erec_t));
        delta_v_time_all.push_back(move(delta_v_time));
        delta_v_final.push_back(delta_v_accum);
    }

    // Compute percentiles and medians
    vector<double> Pdyn_med = median_vector(Pdyn_all);
    vector<double> Pdyn_p10 = percentile_vector(Pdyn_all, 10.0);
    vector<double> Pdyn_p90 = percentile_vector(Pdyn_all, 90.0);

    vector<double> Erec_med = median_vector(Erec_all);
    vector<double> Erec_p10 = percentile_vector(Erec_all, 10.0);
    vector<double> Erec_p90 = percentile_vector(Erec_all, 90.0);

    vector<double> dv_med = median_vector(delta_v_time_all);
    vector<double> dv_p10 = percentile_vector(delta_v_time_all, 10.0);
    vector<double> dv_p90 = percentile_vector(delta_v_time_all, 90.0);

    // Write CSV outputs (convert units where helpful)
    write_time_csv("p_dyn_percentiles.csv", t,
                   [&](){
                       vector<double> tmp = Pdyn_med; for (auto &x: tmp) x *= 1e9; return tmp;
                   }(),
                   [&](){
                       vector<double> tmp = Pdyn_p10; for (auto &x: tmp) x *= 1e9; return tmp;
                   }(),
                   [&](){
                       vector<double> tmp = Pdyn_p90; for (auto &x: tmp) x *= 1e9; return tmp;
                   }(),
                   "P_dyn_nPa");

    write_time_csv("e_rec_percentiles.csv", t,
                   [&](){
                       vector<double> tmp = Erec_med; for (auto &x: tmp) x *= 1e3; return tmp;
                   }(),
                   [&](){
                       vector<double> tmp = Erec_p10; for (auto &x: tmp) x *= 1e3; return tmp;
                   }(),
                   [&](){
                       vector<double> tmp = Erec_p90; for (auto &x: tmp) x *= 1e3; return tmp;
                   }(),
                   "E_rec_mVpm");

    // delta-v percentiles (output in mm/s)
    vector<double> dv_med_mm = dv_med; for (auto &x: dv_med_mm) x *= 1e3;
    vector<double> dv_p10_mm = dv_p10; for (auto &x: dv_p10_mm) x *= 1e3;
    vector<double> dv_p90_mm = dv_p90; for (auto &x: dv_p90_mm) x *= 1e3;
    write_delta_v_time_percentiles("delta_v_time_percentiles.csv", t, dv_med_mm, dv_p10_mm, dv_p90_mm);

    // delta-v histogram (m/s)
    write_delta_v_hist_csv("delta_v_hist.csv", delta_v_final);

    // Print summary stats
    vector<double> dv_sorted = delta_v_final;
    sort(dv_sorted.begin(), dv_sorted.end());
    double median_dv = dv_sorted[dv_sorted.size()/2];
    double dv_p10_val = dv_sorted[max(0, (int)floor(0.10*dv_sorted.size()))];
    double dv_p90_val = dv_sorted[min((int)dv_sorted.size()-1, (int)floor(0.90*dv_sorted.size()))];

    cout << "Delta-V (m/s) median = " << median_dv << "\n";
    cout << "Delta-V (m/s) 10th-90th = " << dv_p10_val << " - " << dv_p90_val << "\n";

    cout << "Wrote CSVs: p_dyn_percentiles.csv, e_rec_percentiles.csv, delta_v_time_percentiles.csv, delta_v_hist.csv\n";
    cout << "Plot with Python (matplotlib) or open CSVs in your plotting tool.\n";
    return 0;
}


SyntaxError: invalid syntax (ipython-input-1516385276.py, line 1)